<a href="https://colab.research.google.com/github/rezamohammadloo/Surface_anomaly_detection_using_unsupervised_algorithms/blob/main/One_Class_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive to access files stored in my Google Drive account
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
# Define directories
normal_dir  = '/content/gdrive/MyDrive/glass-defect-sample/good'
anomaly_dir = '/content/gdrive/MyDrive/glass-defect-sample/anomaly'
mask_dir    = '/content/gdrive/MyDrive/glass-defect-sample/mask'

In [4]:
# Function to load images from directory
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            img = load_img(os.path.join(directory, filename), target_size=(128, 128))
            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img_array)
    return np.array(images)

In [5]:
# Load images
normal_images  =  load_images_from_directory( normal_dir)
anomaly_images =  load_images_from_directory(anomaly_dir)
mask_images    =  load_images_from_directory(  mask_dir )

In [6]:
# Flatten the images
X_flat = normal_images.reshape(normal_images.shape[0], -1)

In [7]:
# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat)



In [8]:
# *******
# # Define the parameter grid
# # param_grid = {
# #     'n_estimators': [100, 200, 300],
# #     'max_samples': [128, 256, 512, 'auto'],
# #     'contamination': [0.01, 0.05, 0.1],
# #     'max_features': [0.5, 0.75, 1.0],
# #     'bootstrap': [True, False]
# # }

# param_grid = {
#     'n_estimators': [100],
#     'max_samples': ['auto'],
#     'contamination': [0.01, 0.05, 0.1],
#     'max_features': [0.5, 0.75, 1.0],
#     'bootstrap': [False]
# }

# # Initialize the Isolation Forest model
# isolation_forest = IsolationForest(random_state=42)

# # Initialize the GridSearchCV object
# grid_search = GridSearchCV(estimator=isolation_forest, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1)

# grid_search.fit(X_good)

# # Print the best parameters and the corresponding score
# print("Best parameters found: ", grid_search.best_params_)
# print("Best AUCROC score: ", grid_search.best_score_)


# all_images = np.concatenate((X_good, X_anomaly), axis=0)
# all_labels = np.concatenate((np.ones(X_good.shape[0]) , -1*np.ones(X_anomaly.shape[0])))

# predictions = isolation_forest.predict(all_images)

In [9]:
# Train One-Class SVM
ocsvm = OneClassSVM(kernel='rbf', nu=0.1)  # Adjust parameters as needed
ocsvm.fit(X_scaled)


OneClassSVM(nu=0.1)

In [10]:
# # Predict anomalies
anomaly_scaled = anomaly_images.reshape(anomaly_images.shape[0], -1)
anomaly_scaled = scaler.transform(anomaly_scaled)
# anomaly_predictions = ocsvm.predict(anomaly_scaled)

In [11]:
all_images = np.concatenate((X_scaled, anomaly_scaled), axis=0)
all_labels = np.concatenate((np.ones(X_scaled.shape[0]) , -1*np.ones(anomaly_scaled.shape[0])))

In [12]:
all_prediction = ocsvm.predict(all_images)

In [13]:
from sklearn.metrics import roc_auc_score, average_precision_score
# Calculate AUC-ROC
auc_roc = roc_auc_score(all_labels, all_prediction)
print(f"AUC-ROC: {auc_roc}")

# Calculate Average Precision (AP)
average_precision = average_precision_score(all_labels, all_prediction)
print(f"Average Precision (AP): {average_precision}")


AUC-ROC: 0.9522292993630573
Average Precision (AP): 0.9806007307058101
